# 0.1.0 Load Data
* Codex Data obtained from: http://welikesharingdata.blob.core.windows.net/forshare/index.html
* Manuscript: https://www.cell.com/cell/pdf/S0092-8674(18)30904-8.pdf

In [1]:
import pandas as pd
df = {}

In [2]:
from clustergrammer2 import net

>> clustergrammer2 backend version 0.4.0


In [3]:
import ipywidgets as widgets
import numpy as np
from bqplot import pyplot as plt
import bqplot

In [4]:
from copy import deepcopy

In [5]:
from glob import glob

### Load X-shift cluster ID

In [6]:
df['cell_type'] = pd.read_excel('../data/big_data/mmc2__codex_cell_type_info.xlsx', index_col=0)
df['cell_type'].shape

(58, 1)

In [7]:
ct_dict = {}
cell_types = []
for inst_id in df['cell_type'].index.tolist():
    inst_ct = df['cell_type']['Imaging phenotype (cell type)'].loc[inst_id]
    cell_types.append(inst_ct)
    ct_dict[inst_id] = inst_ct

In [8]:
cell_types = sorted(list(set(cell_types)))
print(len(cell_types))
cell_types

28


['B cells',
 'B220(+) DN T cells',
 'CD106(+)CD16/32(+)CD31(+) stroma',
 'CD106(+)CD16/32(+)CD31(-)Ly6C(-) stroma',
 'CD106(+)CD16/32(-)Ly6C(+)CD31(+)',
 'CD106(-)CD16/32(+)Ly6C(+)CD31(-)',
 'CD106(-)CD16/32(-)Ly6C(+)CD31(+) stroma',
 'CD11c(+) B cells',
 'CD3(+) other markers (-)',
 'CD31(hi) vascular',
 'CD4(+) T cells',
 'CD4(+)CD8(-)cDC',
 'CD4(+)MHCII(+)',
 'CD4(-)CD8(+)cDC',
 'CD4(-)CD8(-) cDC',
 'CD8(+) T cells',
 'ERTR7(+) stroma',
 'F4/80(+) mphs',
 'FDCs',
 'NK cells',
 'capsule',
 'dirt',
 'erythroblasts',
 'granulocytes',
 'marginal zone mphs',
 'megakaryocytes',
 'noid',
 'plasma cells']

In [9]:
x_dim = 1200
y_dim = 1000

## Check available Files

In [10]:
glob('../data/big_data/*.txt')

['../data/big_data/channelNames_BALBcMRLdataset.txt']

In [11]:
glob('../data/big_data/*.csv')

['../data/big_data/CODEX_MRLdataset_neighborhood_graph.csv',
 '../data/big_data/BALBcMRLdataset_exposuretimes.csv',
 '../data/big_data/Suppl.Table2.CODEX_paper_MRLdatasetexpression.csv']

### Expression Data
See http://welikesharingdata.blob.core.windows.net/forshare/index.html

* CSV file contains mean marker intensities and X, Y, Z corrdinates for each cell relative to the top left corner of its tile. 
* Tile IDs correspond to tile images sorted in the alphanumeric order
* Size parameter corresponds to the cell object sizes in voxels
* "imaging phenotype Cluster ID" column specifies the phenotypic cluster identity as determined by X-shift
* "niche Cluster ID" column specifies the i-niche cluster determined by K-means (K=100)
* neighborhood graph files describes which pairs of cells are adjacent to one another

### Add X-Y position ID to the Cells
This ID will be used to generate the neighborhood graph.

In [137]:
%%time
df['exp'] = pd.read_csv('../data/big_data/Suppl.Table2.CODEX_paper_MRLdatasetexpression.csv')

CPU times: user 9.07 s, sys: 547 ms, total: 9.61 s
Wall time: 9.31 s


In [138]:
df['exp'].shape

(734101, 38)

In [219]:
%%time
# new_rows = ['C-' + str(x) for x in df['exp'].index.tolist()]

new_rows = []
for inst_row in df['exp'].index.tolist():
    inst_ser = df['exp'].loc[inst_row]
    inst_x = inst_ser['X.X']
    inst_y = inst_ser['Y.Y']
    inst_tile = inst_ser['sample_Xtile_Ytile']
    
    new_row = 'C-' + str(inst_row) + '_' + inst_tile + '_' + str(inst_x) + '_' + str(inst_y)
    # new_row = inst_tile + '_' + str(inst_x) + '_' + str(inst_y)
    new_rows.append(new_row)

df['exp'].index = new_rows
print(df['exp'].shape)

KeyboardInterrupt: 

In [220]:
rows = df['exp'].index.tolist()
print(len(rows))
print(len(list(set(rows))))

734101


KeyboardInterrupt: 

In [ ]:
df['exp'].index.tolist()[0]

In [ ]:
new_row

In [ ]:
df['exp'].head()

In [ ]:
df['exp'].shape

### Add Sample Column

In [ ]:
sample_list = [x.split('_')[0] for x in list(df['exp']['sample_Xtile_Ytile'].get_values())]
ser_sample = pd.Series(sample_list, name='sample_slide', index=df['exp'].index.tolist())

In [ ]:
print(len(sorted(list(set(sample_list)))))
list_slides = sorted(list(set(sample_list)))
list_slides

In [ ]:
df['exp']['sample_slide'] = ser_sample

In [ ]:
ser_sample.value_counts()

In [ ]:
cols = df['exp'].columns.tolist()
cols

In [ ]:
exp_cols = cols[1:30]
exp_cols

In [ ]:
unique_dict = {}
for inst_col in cols:
    inst_list_unique = list(df['exp'][inst_col].unique())
    unique_dict[inst_col] = inst_list_unique    
    inst_num_unique = len(inst_list_unique)
    print(inst_col, inst_num_unique)

### Select Single Image Tile
BALBc: normal tissue
MRL/lpr: spleen from animals with systemic autoimmune disease

Start with: 'BALBc-1_X01_Y01'

In [ ]:
df_list = []

all_tiles = ['BALBc-1_X01_Y01', 'BALBc-1_X02_Y01', 'BALBc-1_X01_Y02', 'BALBc-1_X02_Y02']
for inst_tile in all_tiles:
# for inst_tile in ['BALBc-1_X01_Y01', 'BALBc-1_X02_Y01']:
# for inst_tile in ['BALBc-1_X01_Y02', 'BALBc-1_X02_Y02']:    
    keep_rows = []

    ser_tile = df['exp']['sample_Xtile_Ytile']
    ser_found = ser_tile[ser_tile == inst_tile]
    
    keep_rows.extend(ser_found.index.tolist())
    inst_df = df['exp'].loc[keep_rows].transpose()
    print('inst_df', inst_df.shape)
    
    if 'X02' in inst_tile:
        inst_df.loc['X.X'] = inst_df.loc['X.X'] + 1350

    if 'Y01' in inst_tile:        
        inst_df.loc['Y.Y'] = 2000 - inst_df.loc['Y.Y']
    
    if 'Y02' in inst_tile:
        inst_df.loc['Y.Y'] = 1000 - inst_df.loc['Y.Y'] 
        
    df_list.append(inst_df)
    
    
# df['tile'] = df['exp'].loc[keep_rows].transpose()
df['tile'] = pd.concat(df_list, axis=1)
print(df['tile'].shape)

cats = df['tile'].loc['Imaging phenotype cluster ID']
cats = [ct_dict[x] for x in cats]

new_cols = []

cols = df['tile'].columns.tolist()
for index in range(len(cols)):
    
    new_col = (cols[index], 'Cell Type: ' + str(cats[index]))
    new_cols.append(new_col)
df['tile'].columns = new_cols
df['tile'].shape

### Plot Tile Expression Levels

In [ ]:
df['tile-exp-ini'] = df['tile'].loc[exp_cols]
df['tile-exp-ini'].shape

### Set Negative Expression Levels to Zero

In [ ]:
df['tile-exp'] = df['tile-exp-ini']
df['tile-exp'][df['tile-exp'] < 0] = 0

In [ ]:
df['tile-exp'].transpose().describe()

In [ ]:
df['tile-exp'][df['tile-exp'] > 5000] = 5000

In [ ]:
df['tile-exp'].shape

In [ ]:
df['tile'].loc['Z.Z'].head()

### Location of Cells

In [ ]:
df['tile-loc'] = df['tile'].loc[['X.X', 'Y.Y']].transpose()
df['tile-loc'].shape

In [ ]:
# df['tile-loc']['Y.Y'] = 1000 - df['tile-loc']['Y.Y'] 

In [ ]:
df['tile-loc'] = df['tile-loc'].astype('int')

In [ ]:
def set_expression_opacity(inst_gene):
    ser_opacity = df['tile-exp'].loc[inst_gene]
    list_opacity = [float(x/ser_opacity.max()) for x in list(ser_opacity.get_values())]
    scatter.default_opacities = list_opacity

In [ ]:
fig = plt.figure(title='Scatter')
def_tt = bqplot.Tooltip(fields=['name'], formats=[''])

scatter = plt.scatter(df['tile-loc']['X.X'], 
                      df['tile-loc']['Y.Y'], 
                      figsize=(20,10), 
                      ylim=(0,1000), 
                      xlim=(0,1000), stroke='black', 
                      tooltip=def_tt, 
                      names=df['tile-loc'].index.tolist(), 
                      display_names=False)

inst_width = 900
fig.layout.min_height = str(inst_width/1.2) + 'px'
fig.layout.min_width = str(inst_width) + 'px'

set_expression_opacity('NKp46')

scatter.default_size = 100
scatter.colors = ['red']

### Using RGB to HEX
https://www.rgbtohex.net/

In [ ]:
cat_colors = {}
# A 
cat_colors['NK cells'] = '#FB0006'
# B
cat_colors['granulocytes'] = '#FA1400'
# C 
cat_colors['CD4(-)CD8(-) cDC'] = '#FC4B08'
# D 
cat_colors['B220(+) DN T cells'] = '#FD8007'
# E 
cat_colors['plasma cells'] = '#FDBA0A'
# F 
cat_colors['F4/80(+) mphs'] = '#FFF80B'
# G 
cat_colors['FDCs'] = '#FC9CA0'
# H
cat_colors['CD11c(+) B cells'] = '#99FF06'
# I 
cat_colors['capsule'] = '#68FF0A'
# J 
cat_colors['marginal zone mphs'] = '#0A4600'
# K 
cat_colors['noid'] = '#25FF04'
# L 
cat_colors['B cells'] = '#FFFF09'
# M
cat_colors['erythroblasts'] = '#1FFF3C'
# N
cat_colors['CD106(+)CD16/32(+)CD31(+) stroma'] = '#23FF6D'
# O
cat_colors['CD4(-)CD8(+)cDC'] = '#23FFA3'
# P 
cat_colors['CD106(-)CD16/32(-)Ly6C(+)CD31(+) stroma'] = '#20FFDD'
# Q
cat_colors['megakaryocytes'] = '#1CE5FF'
# R 
cat_colors['CD106(-)CD16/32(+)Ly6C(+)CD31(-)'] = '#15A7FF'
# S
cat_colors['CD4(+) T cells'] = '#0D6FFF'
# T
cat_colors['CD4(+)MHCII(+)'] = '#0137FF'
# Ud
cat_colors['CD31(hi) vascular'] = '#0000FF'
# V
cat_colors['CD3(+) other markers (-)'] = '#0700FF'
# W
cat_colors['CD106(+)CD16/32(+)CD31(-)Ly6C(-) stroma'] = '#2D00FF'
# X
cat_colors['CD8(+) T cells'] = '#5900FF'
# Y
cat_colors['ERTR7(+) stroma'] = '#8C00FE'
# Z
cat_colors['CD106(+)CD16/32(-)Ly6C(+)CD31(+)'] = '#C300FF'
# [
cat_colors['CD4(+)CD8(-)cDC'] = '#FB00FA'
# dirt
cat_colors['dirt'] = '#BCBCBC'

### Expression Levels

In [165]:
keep_cols = [x for x in df['tile-exp'].columns.tolist() if 'dirt' not in x[1]]
df['tile-exp-clean'] = df['tile-exp'][keep_cols]
df['tile-exp-clean'].shape

(29, 5018)

In [166]:
net.load_df(df['tile-exp-clean'])
net.set_cat_colors(axis='col', cat_colors=cat_colors, cat_index=1, cat_title='Cell Type')

In [167]:
net.load_df(df['tile-exp-clean'])
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "CD45", "ini": 29, "clust": 17, "rank": 28, "rankvar": 25, "gro…

### Voronoi Plot

In [168]:
from scipy.spatial import Voronoi
vor = Voronoi(df['tile-loc'])

In [169]:
point_list = df['tile-loc'].index.tolist()
point_names = [x[0] for x in point_list]
cat_names   = [x[1].split(': ')[1] for x in point_list]

In [170]:
patch_data = {}
patch_data['x'] = []
patch_data['y'] = []
patch_data['colors'] = []
region_labels = []

region_point_dict = {}
for point_index in range(vor.point_region.shape[0]):
    region_index = vor.point_region[point_index]
    region_point_dict[region_index] = point_index

for region_index in range(len(vor.regions)):
    
    inst_region = vor.regions[region_index]

    if -1 not in inst_region and len(inst_region) > 0:

        point_index = region_point_dict[region_index]
        point_cat = cat_names[point_index]
        region_labels.append(point_cat)
        
        # save cat_colors
        inst_color = cat_colors[point_cat]
        patch_data['colors'].append(inst_color)
        
        x_list = []
        y_list = []
        for inst_vertex in inst_region:
            inst_pos = vor.vertices[inst_vertex]
            x_list.append(inst_pos[0])
            y_list.append(inst_pos[1])
            
        patch_data['x'].append(x_list)
        patch_data['y'].append(y_list)    

In [171]:
import bqplot.pyplot as plt
fig = plt.figure(animation_duration=1000)
patch = plt.plot([], [], 
                 fill='inside',
                 fill_colors=patch_data['colors'],
                 stroke_width=1,
                 close_path=True,
                 labels=region_labels,
                 tooltip=def_tt,
                 axes_options={'x': {'visible': False}, 'y': {'visible': False}},
                )

scatter = plt.scatter(df['tile-loc']['X.X'], 
                      df['tile-loc']['Y.Y'],
#                       tooltip=def_tt, 
                      names=point_names,
                      display_names=False, default_size=2)


inst_width = 950
fig.layout.min_height = str(inst_width/(1.15)) + 'px'
fig.layout.min_width  = str(inst_width) + 'px'

patch.x = patch_data['x']
patch.y = patch_data['y']

plt.xlim(0, 2.0*x_dim)
plt.ylim(0, 2.0*y_dim)

fig
# plt.show()

Figure(animation_duration=1000, axes=[Axis(scale=LinearScale(max=2400.0, min=0.0), visible=False), Axis(orient…

In [172]:
def mouseover_highlight(self, target):
    # print('cat name', target['data']['name'])
    list_opacities = []
    for inst_label in region_labels:
        inst_opacity = 0.25
        if inst_label == target['data']['name']:
            inst_opacity = 1
        list_opacities.append(inst_opacity)

    self.opacities = list_opacities

In [173]:
def reset_highlight(self, target):
#     print('CLICKING')
    list_opacities = [1 for x in region_labels]
    self.opacities = list_opacities

In [174]:
patch.on_hover(mouseover_highlight)
patch.on_element_click(reset_highlight)

In [175]:
cells = df['tile-loc'].index.tolist()
len(cells)

5146

In [177]:
df['tile-loc'].transpose()

,"(C-643_BALBc-1_X01_Y01_10_277, Cell Type: B cells)","(C-1264_BALBc-1_X01_Y01_1006_490, Cell Type: B cells)","(C-1589_BALBc-1_X01_Y01_101_504, Cell Type: B cells)","(C-1611_BALBc-1_X01_Y01_101_759, Cell Type: B cells)","(C-1873_BALBc-1_X01_Y01_1013_254, Cell Type: B cells)","(C-1877_BALBc-1_X01_Y01_1013_293, Cell Type: B cells)","(C-1935_BALBc-1_X01_Y01_1013_914, Cell Type: B cells)","(C-2478_BALBc-1_X01_Y01_1020_25, Cell Type: B cells)","(C-2866_BALBc-1_X01_Y01_1025_128, Cell Type: B cells)","(C-2988_BALBc-1_X01_Y01_1026_609, Cell Type: B cells)",...,"(C-250718_BALBc-1_X02_Y02_525_10, Cell Type: NK cells)","(C-251502_BALBc-1_X02_Y02_1008_129, Cell Type: noid)","(C-252813_BALBc-1_X02_Y02_68_787, Cell Type: noid)","(C-252972_BALBc-1_X02_Y02_787_592, Cell Type: noid)","(C-253114_BALBc-1_X02_Y02_869_585, Cell Type: noid)","(C-253125_BALBc-1_X02_Y02_871_770, Cell Type: noid)","(C-253249_BALBc-1_X02_Y02_95_418, Cell Type: noid)","(C-254494_BALBc-1_X02_Y02_399_712, Cell Type: CD4(+)MHCII(+))","(C-254542_BALBc-1_X02_Y02_590_401, Cell Type: CD4(+)MHCII(+))","(C-254598_BALBc-1_X02_Y02_79_540, Cell Type: CD4(+)MHCII(+))"
X.X,10,1006,101,101,1013,1013,1013,1020,1025,1026,...,1875,2358,1418,2137,2219,2221,1445,1749,1940,1429
Y.Y,1723,1510,1496,1241,1746,1707,1086,1975,1872,1391,...,990,871,213,408,415,230,582,288,599,460


In [178]:
from scipy.spatial.distance import pdist, squareform

In [179]:
mat_cd = pdist(df['tile-loc'], metric='euclidean')

In [180]:
dist_mat = squareform(mat_cd)
dist_mat.shape

(5146, 5146)

In [181]:
df['dist'] = pd.DataFrame(data=dist_mat, index=cells, columns=cells)

In [182]:
df['dist'].head()

,"(C-643_BALBc-1_X01_Y01_10_277, Cell Type: B cells)","(C-1264_BALBc-1_X01_Y01_1006_490, Cell Type: B cells)","(C-1589_BALBc-1_X01_Y01_101_504, Cell Type: B cells)","(C-1611_BALBc-1_X01_Y01_101_759, Cell Type: B cells)","(C-1873_BALBc-1_X01_Y01_1013_254, Cell Type: B cells)","(C-1877_BALBc-1_X01_Y01_1013_293, Cell Type: B cells)","(C-1935_BALBc-1_X01_Y01_1013_914, Cell Type: B cells)","(C-2478_BALBc-1_X01_Y01_1020_25, Cell Type: B cells)","(C-2866_BALBc-1_X01_Y01_1025_128, Cell Type: B cells)","(C-2988_BALBc-1_X01_Y01_1026_609, Cell Type: B cells)",...,"(C-250718_BALBc-1_X02_Y02_525_10, Cell Type: NK cells)","(C-251502_BALBc-1_X02_Y02_1008_129, Cell Type: noid)","(C-252813_BALBc-1_X02_Y02_68_787, Cell Type: noid)","(C-252972_BALBc-1_X02_Y02_787_592, Cell Type: noid)","(C-253114_BALBc-1_X02_Y02_869_585, Cell Type: noid)","(C-253125_BALBc-1_X02_Y02_871_770, Cell Type: noid)","(C-253249_BALBc-1_X02_Y02_95_418, Cell Type: noid)","(C-254494_BALBc-1_X02_Y02_399_712, Cell Type: CD4(+)MHCII(+))","(C-254542_BALBc-1_X02_Y02_590_401, Cell Type: CD4(+)MHCII(+))","(C-254598_BALBc-1_X02_Y02_79_540, Cell Type: CD4(+)MHCII(+))"
"(C-643_BALBc-1_X01_Y01_10_277, Cell Type: B cells)",0.000000,1018.520987,244.560831,490.515035,1003.263674,1003.127609,1188.182646,1040.963016,1025.878160,1068.868561,...,2003.874747,2497.800633,2064.597782,2500.670710,2567.205679,2667.877433,1833.331939,2254.627685,2233.444873,1899.665760
"(C-1264_BALBc-1_X01_Y01_1006_490, Cell Type: B cells)",1018.520987,0.000000,905.108281,944.132406,236.103791,197.124326,424.057779,465.210705,362.498276,120.668969,...,1012.699857,1495.401284,1360.864799,1579.102593,1634.134021,1764.830020,1026.598753,1430.151391,1304.713378,1132.002208
"(C-1589_BALBc-1_X01_Y01_101_504, Cell Type: B cells)",244.560831,905.108281,0.000000,255.000000,945.644754,936.090273,999.921997,1036.340678,997.573055,930.940385,...,1844.752558,2341.938086,1838.634820,2308.471356,2377.916104,2469.241989,1625.340580,2043.322784,2046.101171,1684.304011
"(C-1611_BALBc-1_X01_Y01_101_759, Cell Type: B cells)",490.515035,944.132406,255.000000,0.000000,1042.482134,1024.158191,925.077835,1176.144974,1118.899906,937.083241,...,1791.668775,2287.126800,1670.710328,2199.814765,2273.367546,2348.727528,1496.869066,1903.710325,1947.841113,1540.631364
"(C-1873_BALBc-1_X01_Y01_1013_254, Cell Type: B cells)",1003.263674,236.103791,945.644754,1042.482134,0.000000,39.000000,660.000000,229.106962,126.570139,355.237948,...,1146.551351,1604.571594,1585.595787,1747.461015,1796.106066,1938.432356,1241.579639,1633.236052,1474.767100,1351.610891


In [183]:
# df['dist'][inst_col].hist()

In [184]:
# ser_hood = inst_ser[inst_ser < 100]

In [185]:
# ser_hoodod.shape

In [186]:
num_nn = 10
list_cell_density = []
list_nn_dist = []
for inst_col in df['dist'].columns.tolist():
    # skip self distance of zero
    inst_ser = df['dist'][inst_col].sort_values()[1:]
    
    ser_nn = inst_ser[:num_nn]
    
    # find nearest neighbor cell type distribution
    nn_names = ser_nn.index.tolist()
    nn_cats = [x[1].split(': ')[1] for x in nn_names]
    nn_dist = pd.Series(nn_cats).value_counts()/num_nn
    
    # number of cells in 100px neighborhood
    cell_density = inst_ser[inst_ser < 100].shape[0]
    list_cell_density.append(cell_density)

    nn_dist.name = (inst_col[0], inst_col[1], 'NN CT: ' + nn_dist.index.tolist()[0], 'Density: ' + str(cell_density))
    list_nn_dist.append(nn_dist)

df['nn-dist'] = pd.concat(list_nn_dist, axis=1, sort=True).fillna(0)    

In [187]:
# nn_dist

In [188]:
net.load_df(df['nn-dist'])
net.set_cat_colors(axis='col', cat_colors=cat_colors, cat_index=2, cat_title='NN CT')

# Neighborhood Graph

In [213]:
df['exp'].index.tolist()[0]

'C-0_BALBc-3_X05_Y03_10_70'

In [207]:
df['neighbor'] = pd.read_csv('../data/big_data/CODEX_MRLdataset_neighborhood_graph.csv')
df['neighbor'].shape

(4310498, 8)

In [208]:
df['neighbor'].head()

,Cell1 Sample_Xtile_Ytile,X1,Y1,Cell1 Imaging phenotype cluster ID,Cell2 Sample_Xtile_Ytile,X2,Y2,Cell2 Imaging phenotype cluster ID
0,BALBc-1_X01_Y01,15,221,9619,BALBc-1_X01_Y01,26,243,9596
1,BALBc-1_X01_Y01,15,221,9619,BALBc-1_X01_Y01,41,170,9613
2,BALBc-1_X01_Y01,15,221,9619,BALBc-1_X01_Y01,37,250,9595
3,BALBc-1_X01_Y01,12,444,9615,BALBc-1_X01_Y01,16,367,9613
4,BALBc-1_X01_Y01,12,444,9615,BALBc-1_X01_Y01,12,464,9613


In [ ]:
all_tiles = ['BALBc-1_X01_Y01', 'BALBc-1_X02_Y01', 'BALBc-1_X01_Y02', 'BALBc-1_X02_Y02']

### Make Dictionary of Nieghbor Cells and Neighbor Cell Types

In [242]:
%%time
neighbor_graph = {}
ct_graph = {}
for inst_row in df['neighbor'].index.tolist():
    inst_ser = df['neighbor'].loc[inst_row]
    
    inst_tile = inst_ser['Cell1 Sample_Xtile_Ytile']
    if inst_tile in all_tiles:
        inst_cell = inst_tile + '_' + str(inst_ser['X1']) + '_' + str(inst_ser['Y1'])
    
        inst_neighbor = inst_ser['Cell2 Sample_Xtile_Ytile'] + '_' + str(inst_ser['X2']) + '_' + str(inst_ser['Y2'])
        inst_cat = ct_dict[inst_ser['Cell2 Imaging phenotype cluster ID']]
        
        # add cell to dictionary if necessary
        if inst_cell not in neighbor_graph:
            neighbor_graph[inst_cell] = []
            ct_graph[inst_cell] = []
            
        neighbor_graph[inst_cell].append(inst_neighbor)
        ct_graph[inst_cell].append(inst_cat)

CPU times: user 11min 19s, sys: 1.27 s, total: 11min 20s
Wall time: 11min 22s


### All Cells Neighbor Distribution

In [189]:
net.load_df(df['nn-dist'])
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "B cells", "ini": 27, "clust": 14, "rank": 26, "rankvar": 26, "…

# Merge Expression Levels and Neighborhood Distributions

In [190]:
df['nn-dist'].shape

(27, 5146)

In [191]:
df['nn-dist'].columns.tolist()[0]

('C-643_BALBc-1_X01_Y01_10_277',
 'Cell Type: B cells',
 'NN CT: B cells',
 'Density: 14')

In [192]:
density_dict = {}
nct_dict = {}
for inst_col in df['nn-dist'].columns.tolist():
    nct_dict[inst_col[0]] = inst_col[2]
    density_dict[inst_col[0]] = inst_col[3]

In [193]:
df['tile-exp-clean'].shape

(29, 5018)

### Add Cell Density to Expression Levels

In [194]:
df['exp-common'] = deepcopy(df['tile-exp-clean'])
exp_cols = [(x[0], x[1], nct_dict[x[0]], density_dict[x[0]]) for x in df['exp-common'].columns]
df['exp-common'].columns = exp_cols
df['exp-common'].index = [(x, 'Type: Expression') for x in df['exp-common'].index]
df['exp-common'].shape

(29, 5018)

### Find Common Cells

In [195]:
df['nn-dist-common'] = df['nn-dist'][exp_cols]
df['nn-dist-common'].index = [(x, 'Type: Neighborhood') for x in df['nn-dist-common'].index]
df['nn-dist-common'].shape

(27, 5018)

In [196]:
df['merge'] = pd.concat([df['exp-common'], df['nn-dist-common']], axis=0)
df['merge'].shape

(56, 5018)

In [197]:
df['merge'].columns.tolist()[0]

('C-643_BALBc-1_X01_Y01_10_277',
 'Cell Type: B cells',
 'NN CT: B cells',
 'Density: 14')

# Merge Dist and Expression 

In [198]:
keep_cols = [x for x in df['merge'].columns.tolist() if x[1].split(': ')[1] == 'B cells']
len(keep_cols)

2826

In [199]:
net.load_df(df['exp-common'][keep_cols])
net.filter_threshold(inst_rc='row', num_occur=1, threshold=0.001)
net.normalize(axis='row', norm_type='zscore')
net.clip(-5,5)
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "CD45", "ini": 29, "clust": 19, "rank": 26, "rankvar": 18, "cat…

In [200]:
net.load_df(df['merge'][keep_cols])
net.normalize(axis='row', norm_type='zscore')
net.clip(-5,5)
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "CD45", "ini": 56, "clust": 30, "rank": 52, "rankvar": 44, "cat…

# 'CD4(+) T cells'

In [201]:
keep_cols = [x for x in df['merge'].columns.tolist() if x[1].split(': ')[1] == 'CD4(+) T cells']
len(keep_cols)

378

In [202]:
net.load_df(df['exp-common'][keep_cols])
net.filter_threshold(inst_rc='row', num_occur=1, threshold=0.001)
net.normalize(axis='row', norm_type='zscore')
net.clip(-5,5)
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "CD45", "ini": 29, "clust": 13, "rank": 18, "rankvar": 22, "cat…

In [203]:
net.load_df(df['merge'][keep_cols])
net.filter_threshold(inst_rc='row', num_occur=1, threshold=0.001)
net.normalize(axis='row', norm_type='zscore')
net.clip(-5,5)
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "CD45", "ini": 53, "clust": 31, "rank": 35, "rankvar": 42, "cat…

# Marginal zone mphs

In [204]:
keep_cols = [x for x in df['merge'].columns.tolist() if x[1].split(': ')[1] == 'marginal zone mphs']
len(keep_cols)

151

In [205]:
net.load_df(df['merge'][keep_cols])
net.filter_threshold(inst_rc='row', num_occur=1, threshold=0.001)
net.normalize(axis='row', norm_type='zscore')
net.clip(-5,5)
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "CD45", "ini": 50, "clust": 23, "rank": 35, "rankvar": 31, "cat…

In [ ]:
ser_density = pd.Series(list_cell_density)
ser_density.hist()

In [ ]:
nn_names = ser_nn.index.tolist()
nn_cats = [x[1].split(': ')[1] for x in nn_names]
nn_dist = pd.Series(nn_cats).value_counts()/num_nn
nn_dist

In [ ]:
inst_ser.head()

In [ ]:
inst_ser[1:].head()

In [ ]:
print(cells[0])
df['dist'][cells[0]].sort_values()